# 0.0 Imports

In [1]:
import pandas as pd
import inflection

## 0.2. Loading data

In [2]:
df_sales_0 = pd.read_csv('data/train.csv', low_memory=False)
df_store_0 = pd.read_csv('data/store.csv', low_memory=False)

In [3]:
df_merge = pd.merge(df_sales_0,df_store_0, how='left',on='Store')

In [5]:
df_merge.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [8]:
coluna_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 
            'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

In [11]:
coluna_snake = lambda x: inflection.underscore( x )
coluna_new = list( map( coluna_snake, coluna_old))
df1.columns = coluna_new

# 1.0 Descrição dos dados

In [6]:
df1 = df_merge.copy()

## 1.1. Dimensão dos Dados

In [4]:
print( 'Número de Linhas {}'.format(df_merge.shape[0]))
print( 'Número de Colunas {}'.format(df_merge.shape[1]))

Número de Linhas 1017209
Número de Colunas 18


## 1.2 Data Types

In [14]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

In [15]:
df1['date'] = pd.to_datetime(df1['date'])

In [17]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object